<a href="https://colab.research.google.com/github/AIPythonILP-teamB/AIPythonILP-teamB-Python/blob/kobayashi/%E3%83%A9%E3%83%B3%E3%83%80%E3%83%A0%E3%83%95%E3%82%A9%E3%83%AC%E3%82%B9%E3%83%88%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## データのインポート

In [ ]:
!pip install japanize-matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=e78611f50d2a67faf1c27bb1ba202d53a52d6d822be7b7f8ddd3ac30c9d56f59
  Stored in directory: /root/.cache/pip/wheels/da/a1/71/b8faeb93276fed10edffcca20746f1ef6f8d9e071eee8425fc
Successfully built japanize-matplotlib


In [ ]:
# 数値計算を行うライブラリ numpy をインポート
import numpy as np
# 表データを扱うライブラリ pandas をインポート
import pandas as pd
# 図や表を描画するためのライブラリをインポート
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

In [ ]:
weather_df = pd.read_csv('weather_data_cleaned.csv')
sales_df = pd.read_csv('sales_data_cleaned.csv')

# データ結合
df = pd.merge(sales_df, weather_df, on='日付', how='left')
df = df.drop('曜日_y', axis=1)
#欠損値の削除
df = df.dropna()

## 給料日と祝日を説明変数に



In [ ]:
#日付をdatetimeに変更
df['日付'] = pd.to_datetime(df['日付'], errors='coerce')
!pip install jpholiday
import jpholiday
df['祝日フラグ'] = df['日付'].apply(lambda x: 1 if jpholiday.is_holiday(x) else 0)
df['給料日フラグ'] = df['日付'].dt.day.apply(lambda x: 1 if x in [25,26,27] else 0)
df['金曜日フラグ'] = df['日付'].dt.weekday.apply(lambda x: 1 if x == 4 else 0)

def is_day_before_holiday(date):
    try:
        next_day = date + pd.Timedelta(days=1)
        return 1 if jpholiday.is_holiday(next_day) else 0
    except:
        return 0

# フラグ列の作成
df['祝日前日フラグ'] = df['日付'].apply(is_day_before_holiday)


## ランダムフォレスト

In [ ]:
df.head()

,日付,曜日_x,来客数,総杯数,売上合計(円),ペールエール(本),ペールエール(円),ラガー(本),ラガー(円),IPA(本),...,平均風速(m/s),平均蒸気圧(hPa),平均湿度(％),平均雲量(10分比),最高気温(℃),最低気温(℃),1時間降水量の最大(mm),最大風速(m/s),最小相対湿度(％),気温差
0,2024/4/1,月,16,25,25300,6,6000,5,4000,5,...,2.8,11.9,73.0,6.0,19.7,11.8,2.5,6.3,53.0,7.9
1,2024/4/2,火,19,25,24600,6,6000,4,3200,5,...,3.0,8.2,50.0,5.0,20.6,8.4,0.0,6.1,27.0,12.2
2,2024/4/3,水,11,19,18500,5,5000,4,3200,3,...,1.6,13.0,83.0,10.0,16.6,12.3,3.0,3.9,57.0,4.3
3,2024/4/4,木,6,10,9500,2,2000,2,1600,2,...,2.0,13.6,79.0,10.0,19.9,11.3,3.5,4.7,51.0,8.6
4,2024/4/5,金,10,18,17100,3,3000,5,4000,3,...,2.8,9.6,71.0,10.0,15.1,9.0,0.5,4.9,47.0,6.1


In [ ]:
# 最高気温と最低気温がある場合
df["気温差"] = df["最高気温(℃)"] - df["最低気温(℃)"]

# 日付から曜日や月を生成（'date'列がdatetime型 or 文字列型と仮定）
df["日付"] = pd.to_datetime(df["日付"], errors="coerce")
df["曜日_月"] = (df["日付"].dt.weekday == 0).astype(int)
df["曜日_金"] = (df["日付"].dt.weekday == 4).astype(int)
df["月"] = df["日付"].dt.month


In [ ]:
X = df[['降水量の合計(mm)', '1時間降水量の最大(mm)',
        '気温差', '日照時間(時間)', '平均気温(℃)',
        '曜日_月', '曜日_金','月','平均湿度(％)']]  # 説明変数
y = df['総杯数'].values.ravel()
from sklearn.model_selection import train_test_split

#データを分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# ① モデル定義と学習
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# ② 予測
y_pred = model.predict(X_test)
mean_y = y_test.mean()


# ③ 評価指標
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
relative_mae = mae / mean_y
r2_train = r2_score(y_train, model.predict(X_train))
r2_test = r2_score(y_test, model.predict(X_test))



print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAE / y_test.mean(): {relative_mae:.2%}")
print(f"学習用データの R²: {r2_train:.3f}")
print(f"テスト用データの R²: {r2_test:.3f}")


# ④ 特徴量重要度の可視化
feature_importance = pd.Series(model.feature_importances_, index=X.columns)
feature_importance.sort_values(ascending=True).plot(kind='barh', figsize=(10,6))
plt.title("Feature Importance")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

# ⑤ 実測値と予測値の散布図
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
plt.xlabel("Actual Total Sales (杯数)")
plt.ylabel("Predicted Total Sales (杯数)")
plt.title("Actual vs. Predicted Total Beer Sales")
plt.grid(True)
plt.tight_layout()
plt.show()


KeyError: "['給料日フラグ', '祝日フラグ', '金曜日フラグ'] not in index"

# 各ビールごとの数値

'総杯数', 'ペールエール(本)', 'ラガー(本)', 'IPA(本)', 'ホワイトビール(本)', '黒ビール(本)','フルーツビール(本)'

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# ビールの列
beer_columns = ['ペールエール(本)', 'ラガー(本)', 'IPA(本)', 'ホワイトビール(本)', '黒ビール(本)', 'フルーツビール(本)']

# 結果格納用
results = []

# 各ビールについてモデルを個別に学習・評価
for col in beer_columns:
    y = df[col]
    X = dfX = df[['降水量の合計(mm)', '1時間降水量の最大(mm)',
        '気温差', '日照時間(時間)', '平均気温(℃)',
        '曜日_月', '曜日_金','月','平均湿度(％)']]

    # データ分割
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # モデル学習
    model = RandomForestRegressor(
        n_estimators=150,
        max_depth=5,
        min_samples_split=10,
        min_samples_leaf=5,
        max_features='sqrt',
        random_state=42
    )
    model.fit(X_train, y_train)

    # 予測
    y_pred = model.predict(X_test)

    # 精度指標
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mean_y = y_test.mean()
    relative_mae = mae / mean_y
    r2_train = r2_score(y_train, model.predict(X_train))
    r2_test = r2_score(y_test, y_pred)

    # 結果保存
    results.append({
        'ビール名': col,
        'MAE': round(mae, 2),
        'RMSE': round(rmse, 2),
        '相対MAE': round(relative_mae * 100, 2),  # ％表記
        'テスト用データのR²': round(r2, 3),
        '学習用データのR²': round(r2_train, 3)
    })

# 結果を表示
results_df = pd.DataFrame(results)
display(results_df)


,ビール名,MAE,RMSE,相対MAE,テスト用データのR²,学習用データのR²
0,ペールエール(本),1.43,1.86,28.67,0.509,0.566
1,ラガー(本),1.40,1.90,28.44,0.616,0.680
2,IPA(本),1.06,1.36,29.77,0.524,0.577
3,ホワイトビール(本),1.07,1.55,33.43,0.535,0.637
4,黒ビール(本),1.06,1.43,35.81,0.414,0.560
5,フルーツビール(本),1.25,1.73,47.95,0.273,0.387


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

# 説明変数と目的変数の準備
X = df[['降水量の合計(mm)', '1時間降水量の最大(mm)',
        '気温差', '日照時間(時間)', '平均気温(℃)',
        '曜日_月', '曜日_金','月','平均湿度(％)']]
beer_columns = ['ペールエール(本)', '黒ビール(本)', 'フルーツビール(本)']

# 結果格納用
feature_importances = pd.DataFrame(index=X.columns)

# ビールごとにモデルを学習して特徴量重要度を取得
for beer in beer_columns:
    y = df[beer].values.ravel()
    model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
    model.fit(X, y)
    feature_importances[beer] = model.feature_importances_

# 転置してビール×特徴量の形式に
feature_importances = feature_importances.T

# ヒートマップで可視化
plt.figure(figsize=(12, 6))
sns.heatmap(feature_importances, annot=True, cmap="YlGnBu")
plt.title("ビールごとの特徴量重要度（ランダムフォレスト）")
plt.ylabel("ビールの種類")
plt.xlabel("特徴量")
plt.tight_layout()
plt.show()


KeyError: "['気温差', '曜日_月', '曜日_金', '月'] not in index"